<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e5b5954c8c82db608ecac342b474eb80168d1cb6f0d6c35cb7b412bcbad98928
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 09:29:53
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: -18.85 K (-0.13%)
Current PnL: -21.20 L (-13.89%)
CY Booked + Current PnL: -7.24 L (-4.74%)
-------------------
Total profit:  1.64 L
Total loss:  -22.84 L
-------------------
Total Booked + Current PnL: 19.74 L (15.72%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.54 L (62.59%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,148.88,82.0,M-SC,1.93,163023.0,-11536.0,11575.0,-0.85,-6.61,7.10,0.02,234.0,-1.00,1.13,41.80,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,49.0,M-SC,3.72,86506.0,-14271.0,14360.0,0.07,-14.16,16.60,0.09,253.0,-0.99,0.60,13.65,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.83,89070.0,-1608.0,16514.0,0.08,-1.77,18.54,16.44,63.0,-0.10,0.62,18.04,X40,ATH,PHARMA
84,VOLTAS,1530.00,-1.77,54.0,X-MC,3.71,208575.0,16833.0,20920.0,0.46,8.78,10.03,19.69,78.0,0.80,1.44,16.17,XY25,NTT,AC
35,ICICIGI,2252.93,-15.55,55.0,X-MC,7.53,187888.0,13568.0,21626.0,-0.27,7.78,11.51,20.19,68.0,0.63,1.30,24.07,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LTIM,7201.88,2.83,63.0,H-LC,11.35,263430.0,17279.0,60642.0,1.70,7.02,23.02,31.66,43.0,0.28,1.82,46.17,X200,ATH,IT
69,SONACOMS,806.63,-32.90,65.0,M-SC,6.72,86791.0,-14368.0,54366.0,1.51,-14.20,62.64,39.54,220.0,-0.26,0.60,22.38,AR,ATH,AUTO
41,INFY,2275.00,-19.96,54.0,X-LC,3.60,321161.0,8273.0,163407.0,1.44,2.64,50.88,54.87,2.0,0.05,2.22,11.15,X40,BTT,IT
82,VALIANTORG,838.00,-274.05,37.0,H-SC,12.63,96815.0,-66790.0,188102.0,0.94,-40.82,194.29,74.15,149.0,-0.36,0.67,24.10,XR,NTT,CHEMICALS
13,BSOFT,831.70,-10.71,57.0,H-SC,8.25,97742.0,-42911.0,114339.0,0.91,-30.51,116.98,50.79,151.0,-0.38,0.68,13.80,XR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,105.53,61.0,H-SC,16.01,139818.0,-78678.0,106877.0,-3.04,-36.01,76.44,12.90,161.0,-0.74,0.97,16.26,OX40N,NTT,FINANCE
64,SATIN,274.00,-23.78,55.0,H-SC,2.52,234320.0,-41964.0,185183.0,-1.57,-15.19,79.03,51.83,146.0,-0.23,1.62,14.15,XY24,NTT,FINANCE
58,RAJOOENG,143.10,-48.16,29.0,H-SC,18.17,78680.0,-23820.0,64423.0,-1.39,-23.24,81.88,39.61,135.0,-0.37,0.54,0.00,AR,ATH,MISC
81,VAIBHAVGBL,521.00,64.19,50.0,H-SC,10.79,142579.0,-40196.0,151262.0,-1.37,-21.99,106.09,60.77,159.0,-0.27,0.99,28.40,XR,NTT,JEWELLERY
51,MASFIN,398.61,-17.34,55.0,H-SC,7.19,93975.0,-4005.0,25608.0,-1.28,-4.09,27.25,22.05,164.0,-0.16,0.65,36.60,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.0,35.57,64.0,H-SC,15.58,279072.0,-2051.0,134903.0,-0.49,-0.73,48.34,47.26,171.0,-0.02,1.93,49.52,XY24,NTT,INSURANCE
86,WIPRO,420.0,-15.15,48.0,M-LC,5.21,149767.0,-1178.0,110633.0,0.27,-0.78,73.87,72.51,99.0,-0.01,1.04,5.20,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-19.74,44.0,M-SC,10.09,145991.0,-1653.0,160984.0,-0.57,-1.12,110.27,107.91,216.0,-0.01,1.01,1.85,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,148.88,82.0,M-SC,1.93,163023.0,-11536.0,11575.0,-0.85,-6.61,7.10,0.02,234.0,-1.00,1.13,41.80,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,100.85,49.0,M-SC,3.72,86506.0,-14271.0,14360.0,0.07,-14.16,16.60,0.09,253.0,-0.99,0.60,13.65,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.37,45.0,H-SC,2.00,221193.0,-48474.0,84805.0,-0.49,-17.98,38.34,13.47,158.0,-0.57,1.53,12.47,XY24,NTT,PAINTS
44,JCHAC,2282.00,19986.96,64.0,M-SC,1.44,98118.0,-29687.0,29671.0,-0.01,-23.23,30.24,-0.01,233.0,-1.00,0.68,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,20.80,58.0,M-SC,36.71,163449.0,14229.0,129958.0,-0.63,9.54,79.51,96.62,245.0,0.11,1.13,76.89,XY25,NTT,FINANCE
38,INDIAMART,4810.62,-48.34,47.0,H-SC,4.60,128700.0,5364.0,126268.0,-0.73,4.35,98.11,106.72,139.0,0.04,0.89,28.74,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-48.34,47.0,H-SC,4.60,128700.0,5364.0,126268.0,-0.73,4.35,98.11,106.72,139.0,0.04,0.89,28.74,AR,ATH,MISC
25,FINCABLES,1641.55,-19.74,44.0,M-SC,10.09,145991.0,-1653.0,160984.0,-0.57,-1.12,110.27,107.91,216.0,-0.01,1.01,1.85,OX40N,ATH,CABLES
86,WIPRO,420.00,-15.15,48.0,M-LC,5.21,149767.0,-1178.0,110633.0,0.27,-0.78,73.87,72.51,99.0,-0.01,1.04,5.20,XR,NTT,IT
85,WHIRLPOOL,2270.00,-43.52,34.0,M-SC,7.82,110676.0,-4122.0,98159.0,-0.32,-3.59,88.69,81.92,236.0,-0.04,0.77,31.02,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.85,32.0,H-MC,5.23,204812.0,-4762.0,79426.0,-0.18,-2.27,38.78,35.63,119.0,-0.06,1.42,14.02,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83788.0,-44882.0,90181.0,-0.52,-34.88,107.63,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-21.86,23.0,X-LC,0.30,240647.0,-35763.0,456916.0,-1.05,-12.94,189.87,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
26,GILLETTE,11206.78,-19.05,29.0,X-SC,6.10,250095.0,-3801.0,86108.0,-0.08,-1.50,34.43,32.42,85.0,-0.04,1.73,13.38,X40,ATH,FMCG
27,GLAXO,3466.20,-20.20,29.0,X-MC,7.53,198024.0,-1260.0,79269.0,-0.51,-0.63,40.03,39.15,60.0,-0.02,1.37,28.19,X40,ATH,PHARMA
56,QUESS,424.00,-53.72,30.0,X-SC,37.27,45813.0,-19193.0,47045.0,-0.73,-29.52,102.69,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-21.86,23.0,X-LC,0.30,240647.0,-35763.0,456916.0,-1.05,-12.94,189.87,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,43.0,X-LC,0.91,197849.0,-2289.0,23188.0,-0.31,-1.14,11.72,10.44,5.0,-0.10,1.37,4.36,X40,NTT,FMCG
34,HONAUT,58357.33,-26.21,31.0,X-SC,1.29,104835.0,-23103.0,70239.0,0.33,-18.06,67.00,36.84,90.0,-0.33,0.72,7.33,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.92,60.0,X-MC,1.47,182832.0,6094.0,48633.0,-0.45,3.45,26.60,30.97,75.0,0.13,1.26,16.44,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.97,61.0,H-LC,1.78,161250.0,-24845.0,72321.0,-0.75,-13.35,44.85,25.51,49.0,-0.34,1.11,18.83,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83788.0,-44882.0,90181.0,-0.52,-34.88,107.63,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-53.72,30.0,X-SC,37.27,45813.0,-19193.0,47045.0,-0.73,-29.52,102.69,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
54,PAGEIND,51605.08,-29.69,34.0,X-MC,8.17,156220.0,-7520.0,50193.0,-0.19,-4.59,32.13,26.07,55.0,-0.15,1.08,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-21.86,23.0,X-LC,0.30,240647.0,-35763.0,456916.0,-1.05,-12.94,189.87,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.44,30.0,X-MC,6.92,174168.0,-26338.0,123485.0,-0.36,-13.14,70.90,48.45,64.0,-0.21,1.20,0.44,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-53.72,30.0,X-SC,37.27,45813.0,-19193.0,47045.0,-0.73,-29.52,102.69,42.84,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-42.21,35.0,X-SC,3.09,78791.0,-66369.0,142297.0,-0.55,-45.72,180.60,52.31,91.0,-0.47,0.54,5.47,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.89,23.0,X-SC,10.90,83788.0,-44882.0,90181.0,-0.52,-34.88,107.63,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.73,50.0,X-MC,3.83,89070.0,-1608.0,16514.0,0.08,-1.77,18.54,16.44,63.0,-0.10,0.62,18.04,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.21,31.0,X-SC,1.29,104835.0,-23103.0,70239.0,0.33,-18.06,67.00,36.84,90.0,-0.33,0.72,7.33,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-26.21,60.0,X-LC,8.14,292763.0,-53193.0,119886.0,0.89,-15.38,40.95,19.28,1.0,-0.44,2.02,8.23,X40,ATH,IT
41,INFY,2275.00,-19.96,54.0,X-LC,3.60,321161.0,8273.0,163407.0,1.44,2.64,50.88,54.87,2.0,0.05,2.22,11.15,X40,BTT,IT
77,TMPV,1065.00,-21.86,23.0,X-LC,0.30,240647.0,-35763.0,456916.0,-1.05,-12.94,189.87,152.37,3.0,-0.08,1.66,0.00,XY24,NTT,AUTO
83,VBL,671.64,-18.89,45.0,X-LC,6.11,290705.0,-25137.0,137794.0,-0.49,-7.96,47.40,35.67,4.0,-0.18,2.01,4.79,X40N,ATH,FMCG
43,ITC,452.00,-40.74,43.0,X-LC,0.91,197849.0,-2289.0,23188.0,-0.31,-1.14,11.72,10.44,5.0,-0.10,1.37,4.36,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7627.78,63.0,L-SC,18.05,84779.0,-9031.0,85652.0,-1.17,-9.63,101.03,81.67,271.0,-0.11,0.59,64.18,XR,NTT,CERAMICS
51,MASFIN,398.61,-17.34,55.0,H-SC,7.19,93975.0,-4005.0,25608.0,-1.28,-4.09,27.25,22.05,164.0,-0.16,0.65,36.60,XR,ATH,FINANCE
71,SURYODAY,216.00,53.51,44.0,H-SC,12.60,140403.0,-38668.0,79917.0,-0.35,-21.59,56.92,23.03,167.0,-0.48,0.97,39.31,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,64.19,50.0,H-SC,10.79,142579.0,-40196.0,151262.0,-1.37,-21.99,106.09,60.77,159.0,-0.27,0.99,28.40,XR,NTT,JEWELLERY
66,SHALBY,327.00,1037.16,35.0,M-SC,16.91,154685.0,-26458.0,71929.0,-0.30,-14.61,46.50,25.10,250.0,-0.37,1.07,23.96,XY24,NTT,HEALTHCARE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,148.88,82.0,M-SC,1.93,163023.0,-11536.0,11575.0,-0.85,-6.61,7.10,0.02,234.0,-1.00,1.13,41.80,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.35,83.0,X-LC,10.86,255270.0,3502.0,49242.0,-0.18,1.39,19.29,20.95,31.0,0.07,1.76,37.52,X40,ATH,PAINTS
13,BSOFT,831.70,-10.71,57.0,H-SC,8.25,97742.0,-42911.0,114339.0,0.91,-30.51,116.98,50.79,151.0,-0.38,0.68,13.80,XR,ATH,IT
81,VAIBHAVGBL,521.00,64.19,50.0,H-SC,10.79,142579.0,-40196.0,151262.0,-1.37,-21.99,106.09,60.77,159.0,-0.27,0.99,28.40,XR,NTT,JEWELLERY
40,INDUSINDBK,1800.00,-695.06,66.0,L-MC,5.71,48185.0,-29022.0,54415.0,-0.31,-37.59,112.93,32.89,259.0,-0.53,0.33,32.70,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.59
1,25,44.38
2,50,75.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.72
MC    28.65
LC    24.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.97
X40      20.51
X40N     11.53
XY25     10.17
XR        9.50
AR        9.03
OX40N     7.71
X200      1.82
SR        1.00
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.27
X-MC    22.26
X-LC    19.73
M-SC    13.28
X-SC     7.74
H-MC     4.69
H-LC     2.93
L-SC     1.43
M-MC     1.37
M-LC     1.04
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.61,-4.81,38.70
IT,13.42,-14.82,74.14
FINANCE,10.54,-11.61,64.10
MISC,7.42,-21.17,70.81
PAINTS,6.06,-5.29,20.94
ELECTRICAL,5.63,-10.74,50.00
INSURANCE,4.49,2.71,31.57
PHARMA,3.41,-1.55,35.62
AUTO,3.35,-18.53,132.17


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3277085.0,21
AR,1263782.0,10
XR,1261172.0,13
X40,1000390.0,14
X40N,702601.0,9
OX40N,680896.0,10
XY25,465076.0,7
SR,275578.0,2
MH,66929.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3472233.0,25
M-SC,1433462.0,16
X-LC,1212875.0,11
X-MC,1173625.0,16
X-SC,721571.0,8
H-MC,400340.0,3
L-SC,256078.0,3
H-LC,132963.0,2
M-LC,110633.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1177519.0      6
           AR         851346.0      5
           XR         781110.0      7
M-SC       XY24       760035.0      6
X-LC       XY24       527218.0      2
X-MC       X40        418611.0      7
X-LC       X40        405490.0      5
X-MC       XY24       348879.0      3
H-SC       OX40N      319751.0      4
X-SC       XY24       279821.0      3
H-SC       SR         275578.0      2
M-SC       OX40N      273852.0      5
X-SC       X40N       265461.0      3
X-MC       X40N       230529.0      4
H-MC       AR         216727.0      2
X-LC       X40N       206611.0      2
H-MC       XY24       183613.0      1
X-SC       X40        176289.0      2
X-MC       XY25       175606.0      2
L-SC       XR         168785.0      2
M-SC       XR         146229.0      2
           XY25       129958.0      1
           AR         123388.0      2
M-LC       XR         110633.0      1
L-SC       OX40N       87293.0      1
X-LC       XY25        73556.0      2
H-LC       AR          72321.0      1
H-SC       MH          66929.0      1
H-LC       X200        60642.0      1
L-MC       XR          54415.0      1
M-MC       XY25        47796.0      1
L-LC       XY25        38160.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
